In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:30:10_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0


In [2]:
!nvidia-smi

Wed Apr 17 07:48:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.78                 Driver Version: 551.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
| 35%   38C    P5             22W /  170W |     949MiB /  12288MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Load Pollen DETR from Hugging Face Hub

In [3]:
import torch
import supervision as sv
import transformers
import torch
from transformers import DetrForObjectDetection, DetrImageProcessor, pipeline
import re
import os
import glob
import random
import cv2
import numpy as np
import json
import pandas as pd
import shutil
import os
import torchvision
from torch.utils.data import DataLoader
import random
from label_studio_sdk import Client

# settings
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CHECKPOINT = 'Charliesgt/pollencounter_detr_resnet50' # '_benchmark'
CONFIDENCE_TRESHOLD = 0.3
IOU_TRESHOLD = 0.5
RAW_INFERENCE_DIRECTORY = '../data/raw/USPollen/raw_images/'
INFERENCE_DIRECTORY='../data/raw/USPollen/images/'
INFERENCE_ANNOTATION_DIRECTORY='../data/raw/USPollen/yolo_annotations/'
pipeline_model = pipeline("object-detection", model=CHECKPOINT, device_map="auto")
#image_processor = DetrImageProcessor.from_pretrained(CHECKPOINT)
#model = DetrForObjectDetection.from_pretrained(CHECKPOINT)
#model.to(DEVICE)

## Load Inference Data

In [40]:
df = pd.read_csv('../data/database/pollen_media_metadata.csv')
df = df[df.source=='Kean University']
df['friendly_name'] = df.name.str[4:].str.split('(').str[0].str.strip()
df = df.rename(columns={'common_name_id':'latin_name_id','specific_name':'latin_name','name':"image_name"})

df['count'] = df.groupby('latin_name_id').cumcount() + 1

# Create the new_image_name column using string formatting
df['new_image_name'] = df['latin_name_id'] + '_' + df['count'].astype(str) + '.' + df['media_type']

# Drop the count column if it's no longer needed
df.drop(columns=['count'], inplace=True)
df.to_csv('../data/raw/USPollen/metadata.csv', index=None)

image_rename_dict = dict(zip(list(df.image_name.values), list(df.new_image_name.values)))
import os
import shutil

# Iterate over the image_rename_dict and rename/copy the images
for src_image_name, target_image_name in image_rename_dict.items():
    src_image_path = os.path.join(RAW_INFERENCE_DIRECTORY, src_image_name)
    target_image_path = os.path.join(INFERENCE_DIRECTORY, target_image_name)

    # Check if the source image exists
    if os.path.exists(src_image_path):
        # Copy the image to the target directory with the new name
        shutil.copyfile(src_image_path, target_image_path)
        #print(f"Image '{src_image_name}' copied and renamed to '{target_image_name}'")
    else:
        pass
        #print(f"Image '{src_image_name}' not found in the source directory.")

# Confirm completion
print("Image renaming and copying completed.")

classes_df = df.drop_duplicates(['latin_name_id'])[['latin_name_id','friendly_name']].copy()
classes_df['family'] = classes_df['latin_name_id'].str.split('_').str[0]
classes_df['genus'] = classes_df['latin_name_id'].str.rsplit('_', n=1).str[0]
classes_df.to_csv('../data/raw/USPollen/class_map.csv', index=None)

Image renaming and copying completed.


In [4]:
import random

import cv2

CLASS_COLORS = {
    'JUGLANDACEAE_CARYA_OVATA': '#7a5b07',
    'ULMACEAE_ULMUS_AMERICANA': '#0a3363',
    'CUPRESSACEAE_CUPRESSUS_ARIZONICA': '#2e591b',
    'SALICACEAE_SALIX_LASIOLEPIS': '#8a9a5b',
    'SALICACEAE_SALIX_NIGRA': '#3f3a3a',
    'SAPINDACEAE_ACER_NEGUNDO': '#974600',
    'ASTERACEAE_XANTHIUM_STRUMARIUM': '#6a7f0d',
    'POLYGONACEAE_RUMEX_CRISPUS': '#60341b',
    'PLANTAGINACEAE_PLANTAGO_LANCEOLATA': '#2a533d',
    'ASTERACEAE_AMBROSIA_TRIFIDA': '#5e5a1b',
    'ASTERACEAE_SOLIDAGO_CANADENSIS': '#a08c0c',
    'CANNABACEAE_CELTIS_OCCIDENTALIS': '#4f6821',
    'JUGLANDACEAE_CARYA_TOMENTOSA': '#7b5c1c',
    'AMARANTHACEAE_CHENOPODIUM_ALBUM': '#5b0c4a',
    'POACEAE_DACTYLIS_GLOMERATA': '#314922',
    'PINACEAE_PINUS_STROBUS': '#1c3f1b',
    'SALICACEAE_POPULUS_SP': '#2c515a',
    'ASTERACEAE_AMBROSIA_PSILOSTACHYA': '#0a737a',
    'BETULACEAE_ALNUS_RUBRA': '#583725',
    'MORACEAE_MORUS_RUBRA': '#54101d',
    'CUPRESSACEAE_JUNIPERUS_SCOPULORUM': '#4d8d5b',
    'ASTERACEAE_ARTEMISIA_TRIDENTATA': '#514b1b',
    'ASTERACEAE_ARTEMISIA_PYCNOCEPHALA': '#7e7a0e',
    'JUGLANDACEAE_CARYA_LACINIOSA': '#4d2f0b',
    'PLATANACEAE_PLATANUS_SP': '#225a0e',
    'PLATANACEAE_PLATANUS_RACEMOSA': '#0b3b60',
    'CHENOPODIACEAE_SALSOLA_KALI': '#664a10',
    'JUGLANDACEAE_JUGLANS_CALIFORNICA': '#5a4012',
    'FAGACEAE_FAGUS_GRANDIFOLIA': '#2e3b1b',
    'BETULACEAE_BETULA_NIGRA': '#1d334e',
    'BETULACEAE_BETULA_OCCIDENTALIS': '#542213',
    'CUPRESSACEAE_JUNIPERUS_VIRGINIANA': '#1d5b42',
    'BETULACEAE_ALNUS_INCANA': '#392d14',
    'ASTERACEAE_ARTEMISIA_VULGARIS': '#431a1a',
    'URTICACEAE_URTICA_DIOICA': '#1b541d',
    'FAGACEAE_QUERCUS_AGRIFOLIA': '#3d3a18',
    'OLEACEAE_OLEA_EUROPAEA': '#46591b',
    'CASUARINACEAE_CASUARINA_EQUISETIFOLIA': '#0e5c4a',
}

def convert_voc_to_yolo(voc_bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = voc_bbox
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

class Prediction:
    def __init__(self, task, image_url, result):
        self.task = task
        self.image_url = image_url
        self.result = result



class AutoAnnotator:
    def __init__(self, label_studio_client, project_name, image_dir, n_tasks = None):
        self.ls = label_studio_client
        self.ls.check_connection()
        self.project_name = project_name
        self.project = self._load_project()
        self.unlabeled_tasks = self.get_unlabeled_tasks(n_tasks)
        self.image_dir = image_dir

    def _make_labeling_interface(self):
        labels = [f'''<Label value="{cl}" background="{color}" />''' for cl, color in CLASS_COLORS.items()]
        return f'''
            <View>
              <Image name="image" value="$image" zoom="true"/>
              <Header value="Rectangle Labels"/>
              <RectangleLabels name="label" toName="image">
                {"".join(labels)}
              </RectangleLabels>
            </View>
        '''

    def _load_project(self):
        projects = self.ls.list_projects(**{'title': self.project_name})
        if len(projects) > 0:
            project = projects[0]
        else:
            project = self.ls.start_project(
                title=self.project_name,
                label_config=self._make_labeling_interface(),
            )
        return project

    def get_unlabeled_tasks(self, n_tasks=None):
        unlabeled_tasks_ids = self.project.get_unlabeled_tasks_ids()
        if len(unlabeled_tasks_ids) == 0:
            raise Exception('No Tasks Left To Label')
        if n_tasks is not None:
            unlabeled_tasks_ids = random.sample(unlabeled_tasks_ids, n_tasks)
        return self.project.get_tasks(selected_ids=unlabeled_tasks_ids)

    def get_model_predictions(self, pipeline_model, threshold=0.4):
        image_urls = [f"{self.image_dir}/{task['data']['image'].split('-',1)[1]}" for task in self.unlabeled_tasks]
        print(f'Generating Model Predictions for {len(image_urls)} images...')
        results = pipeline_model(image_urls, threshold=threshold)
        predictions = []
        for task, image_url, result in zip(self.unlabeled_tasks, image_urls, results):
            predictions.append(Prediction(task=task, image_url=image_url, result=result))
        return predictions

    def get_annotations(self, predictions, class_name_override=None, model_version='pollen20_v1'):
        annotations = []
        for prediction in predictions:
            image = cv2.imread(prediction.image_url)
            original_height, original_width, channels = image.shape
            annotation = {
                'result': [],
                'ground_truth':True,
                'task': prediction.task['id'],
            }
            for result in prediction.result:
                voc_bbox = result['box']['xmin'], result['box']['ymin'], result['box']['xmax'], result['box']['ymax']
                yolo_x, yolo_y, yolo_w, yolo_h = convert_voc_to_yolo(voc_bbox, original_width, original_height)

                class_name = result['label'] if class_name_override is None else class_name_override
                annotation['result'].extend([{
                    
                    "source": "$image",
                    "type": "rectanglelabels",
                    "from_name": "label",
                    "to_name": "image",
                    "original_width": original_width,
                    "original_height": original_height,
                    "image_rotation": 0,
                    "value": {
                        "rotation": 0,
                        "x": (yolo_x - yolo_w / 2) * 100,
                        "y": (yolo_y - yolo_h / 2) * 100,
                        "width": yolo_w * 100,
                        "height": yolo_h * 100,
                        "rectanglelabels": [class_name]
                    }
                
                }])
            annotations.append(annotation)
        return annotations


    def auto_run(self, pipeline_model,threshold=0.4, class_name_override=None):
        responses = []
        predictions = self.get_model_predictions(pipeline_model, threshold=threshold)
        annotations = self.get_annotations(predictions, class_name_override=class_name_override)
        #return annotations
        for annotation in annotations:
            task_id = annotation['task']
            res = self.project.create_annotation(task_id, **annotation)
            responses.append(res)
        return responses



PROJECT_TITLE = 'USPollen'
ls = Client(url='https://charliesgt-labelstudio.hf.space', api_key='85d84612906fa5c764b8ea1ab7012b64c85712e5')
ls.check_connection()
aa = AutoAnnotator(label_studio_client=ls, project_name=PROJECT_TITLE, image_dir='../data/raw/ku_pollen/ImagesFromVideos/2020-09-15_100x_Video_scan_1/', n_tasks=None)
annotations = aa.auto_run(pipeline_model=pipeline_model,threshold=0.8, class_name_override='BETULACEAE_BETULA_OCCIDENTALIS')

Generating Model Predictions for 32 images...


In [2]:
import sys
sys.path.append('../')
from services.dataset_builder.process import PollenDataset
from services.dataset_builder.globalpollen.detection import refresh as detection_refresh
ku_pollen_dataset = PollenDataset('../data/raw/ku_pollen/')
#ku_pollen_dataset.load_media(ku_pollen_dataset.root_path+'Videos')
#ku_pollen_dataset.convert_video_to_images(ku_pollen_dataset.root_path+'Videos/data/raw/ku_pollen/Videos/2020-09-15 100x Video scan 1.mp4')
ku_pollen_dataset.convert_ls_annotations_to_master()
detection_refresh()

ls_to_master failed for: FRAME_3_2020-09-15_100x_Video_scan_1.jpg
Refreshing Global Detection Dataset
  -- Images: 2653
  -- Annotations: 10714


In [5]:
def convert_ls_to_yolo(ls_bbox, img_width, img_height):
    x, y, width, height = ls_bbox
    yolo_x = (x + width / 2) / 100
    yolo_y = (y + height / 2) / 100
    yolo_w = width / 100
    yolo_h = height / 100
    return yolo_x, yolo_y, yolo_w, yolo_h

def convert_yolo_to_voc(yolo_bbox, img_width, img_height):
    x_center, y_center, width, height = yolo_bbox
    x_min = int((x_center - width / 2) * img_width)
    y_min = int((y_center - height / 2) * img_height)
    x_max = int((x_center + width / 2) * img_width)
    y_max = int((y_center + height / 2) * img_height)
    return x_min, y_min, x_max, y_max

def convert_ls_to_voc(yolo_bbox, img_width, img_height):
    x, y, width, height = yolo_bbox
    x_min = int(x * img_width / 100)
    y_min = int(y * img_height / 100)
    x_max = int((x + width) * img_width / 100)
    y_max = int((y + height) * img_height / 100)
    return x_min, y_min, x_max, y_max

def convert_yolo_to_ls(yolo_bbox, img_width, img_height):
    yolo_x, yolo_y, yolo_w, yolo_h = yolo_bbox
    x = (yolo_x - yolo_w / 2) * 100
    y = (yolo_y - yolo_h / 2) * 100
    width = yolo_w * 100
    height = yolo_h * 100
    return x, y, width, height


bboxes_df = []
label_studio_annotations = '../data/raw/USPollen/ls_annotations'
for ann_file in glob.glob(label_studio_annotations+'/*.json'):
    ls_annotation = json.load(open(ann_file, "r"))
    for image_annotation in ls_annotation:
        image_name = image_annotation['image'].split('-')[1]

        for label in image_annotation['label']:
            class_name = label['rectanglelabels'][0]
            original_width = label['original_width']
            original_height = label['original_height']
            x, y, w, h = convert_ls_to_yolo(
                (
                    label['x'],
                    label['y'],
                    label['width'],
                    label['height'],
                ),
                original_width,
                original_height
            )
            x1, y1, x2, y2 = convert_yolo_to_voc((x, y, w, h), original_width, original_height)
            bbox = {
                'image_name': image_name,
                'x1': x1,
                'y1': y1,
                'x2': x2,
                'y2': y2,
                'class_name': class_name
            }
            bboxes_df.append(bbox)

bboxes_df = pd.DataFrame(bboxes_df)
bboxes_df.to_csv('../data/raw/USPollen/bboxes.csv',index=None)